# Ejercicio 3

Implemente el algoritmo de entrenamiento de un SOM bidimensional de forma que se pueda ver gráficamente el mapa topológico durante todo el proceso. Para poder observar el ordenamiento topológico, en el mapa grafique líneas de unión entre pares de neuronas vecinas. Para realizar las pruebas de ordenamiento topológico utilice los patrones provistos en los archivos `circulo.csv` y `te.csv`, considerando en estos casos 1000  ́epocas de entrenamiento como máximo. Dichos datos fueron generados aleatoriamente con distribución uniforme dentro de las siguientes figuras planas:

1. un círculo de radio 1 centrado en el origen
2. Una T

Repita el entrenamiento con los datos en T pero para un SOM unidimensional con la misma cantidad de neuronas.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
rng = np.random.default_rng()

In [3]:
def generar_particiones(filename, n, p_test):
    ds = np.genfromtxt(filename, delimiter=',')
    n_test = int(ds.shape[0] * p_test)
    n_train = ds.shape[0] - n_test
    M_test = np.zeros((n,n_test),dtype = int)
    M_train = np.zeros((n, n_train),dtype = int)

    for i in range(n):
        idx=np.random.choice(range(ds.shape[0]),ds.shape[0],replace = False)
        idx_test = idx[0:n_test]
        idx_train = idx[n_test:ds.shape[0]]
        M_test[i] = idx_test
        M_train[i] = idx_train
    
    return (ds, M_test, M_train)

In [86]:
ds, testPartitionsIdx, trainPartitionsIdx  =  generar_particiones('gtp2datos/circulo.csv', 1, 0.2)
NInputs = 2

# Cantidad de patrones para entrenamiento
NPatternsTrain = trainPartitionsIdx.shape[1]

# Generar el vector de entradas para entrenamiento (solo una partición)
X = ds[trainPartitionsIdx[0], :]

In [ ]:
# el mapa es un arreglo cuyos componentes representan las neuronas numeradas fila a fila

nRows = 3                                           # cantidad de filas
nCols = 3                                           # cantidad de columnas
actRatio = 2                                        # radio de activación
learningRate = 1e-3

# inicialización de pesos [habría que hacerlo entre -0.5 y 0.5?]
som = rng.random((nRows * nCols, NInputs))
print(som)

SOM usando un vector: se organiza la numeración de izquierda a derecha, numerando fila a fila

falta el bucle para actualizar los pesos de cada patrón, generar funciones para cada cosa, y hacer las tres etapas bajando los parametros cada vez

In [ ]:

for pattern in X:
    
    #búsqueda de la neurona ganadora
    actN = np.argmin(np.linalg.norm(pattern - som, 1, axis = 1))


    #buscar en qué fila cae la neurona
    idxRow = 0; cRow = 0
    for i in range(0, len(som), nCols):
        if actN in range(i, i+nCols):
            idxRow = cRow
            break
        else:
            cRow += 1

    # Adaptación de pesos
    # wj(n) + η(n) (x(n) − wj(n)) de radio de vecindad

    # en la fila de la neurona activa
    print('neuronas activadas:')
    minIdx = idxRow*nCols
    maxIdx = idxRow*nCols + nCols
    som[np.clip(actN - actRatio, minIdx, maxIdx) : np.clip(actN + actRatio + 1, minIdx, maxIdx)] = \
        som[np.clip(actN - actRatio, minIdx, maxIdx) : np.clip(actN + actRatio + 1, minIdx, maxIdx)] + \
            learningRate * (pattern - som[np.clip(actN - actRatio, minIdx, maxIdx) : np.clip(actN + actRatio + 1, minIdx, maxIdx)])
    print(f'fila 0, ({np.clip(actN - actRatio, idxRow*nCols, idxRow*nCols +nCols)}:{np.clip(actN + actRatio + 1, idxRow*nCols, idxRow*nCols + nCols)}]')

    # en las filas hacia arriba y abajo de la neurona activa
    for i in range(1, actRatio + 1):
        # si la fila anterior existe, su índice no es negativo
        if idxRow - i > -1:
            
            minIdx = (idxRow - i) * nCols
            maxIdx = (idxRow - i) * nCols + nCols
            print(f'rango de indices: {minIdx}-{maxIdx}')
            # clampeo los valores para que se encuentren dentro del rango de índices de la fila
            idxIni = np.clip(actN - (i * nCols) - (actRatio - i), minIdx, maxIdx)
            idxFin = np.clip(actN - (i * nCols) + (actRatio - i) + 1, minIdx, maxIdx)
            print(f'fila - {i}, ({idxIni}:{idxFin}]')
            # actualizo los pesos de la neurona
            som[idxIni : idxFin] = som[idxIni : idxFin] + learningRate * (pattern - som[idxIni : idxFin])
            # si la fila a siguiente existe, el índice es menor que la cantidad de cols del mapa
        if idxRow + i < nRows:
            minIdx = (idxRow + i) * nCols
            maxIdx = (idxRow + i) * nCols + nCols
            print(f'rango de indices: {minIdx}-{maxIdx}')
            # clampeo los valores para que se encuentren dentro del rango de índices de la fila
            idxIni = np.clip(actN + (i * nCols) - (actRatio - i), minIdx, maxIdx)
            idxFin = np.clip(actN + (i * nCols) + (actRatio - i) + 1, minIdx, maxIdx)
            print(f'fila + {i}, ({actN + (i * nCols) - (actRatio - i)}:{actN + (i * nCols) + (actRatio - i) + 1}]')
            som[idxIni : idxFin] = som[idxIni : idxFin] + learningRate * (pattern - som[idxIni : idxFin])
